In [1]:
import matplotlib.pyplot as plt
import numpy as np
from basic.utils.run.train import train
from basic.utils.common_utils import put_data_to_gpu
from basic.utils.run.train import Trainer
from basic.utils.vis_utils import VisualWindow
import easydict
import yaml
import torch
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

/home/ph/ENV/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


读取最高级配置文档，获取TOP config。获取train config

In [2]:
from basic.utils.config_utils import cfg_from_yaml_file
from pprint import pprint
top_cfg_path = '/home/ph/Desktop/PointCloud/utils_my/basic/model/model_cfg/second.yaml'
# with open(top_cfg_path, 'rb') as f:
#     top_cfg = easydict.EasyDict(yaml.load(f))
top_cfg = cfg_from_yaml_file(top_cfg_path, merge_subconfig=True)
train_cfg = top_cfg.TRAIN_CONFIG
pprint(train_cfg)

{'BATCH': 4,
 'CLASS_NAMES': ['Car'],
 'EPOCHS': 150,
 'GRAD_NORM_CLIP': 10,
 'OPTIMIZER': {'AMSGRAD': False,
               'BETAS': [0.9, 0.999],
               'LR': 0.003,
               'LR_SCHEDULER': None,
               'LR_WARMUP': True,
               'NAME': 'AdamW',
               'WARMUP_EPOCH': 1,
               'WARMUP_LR': 1e-05,
               'WEIGHT_DECAY': 0.01},
 'SAVE_STATE': 10}


# Train

In [3]:
# train(train_cfg)

# Inference
 首先读取数据集配置信息，建立测试数据集

In [4]:
pprint(top_cfg.INFERENCE_CONFIG)

{'POST_PROCESSING': {'CONFIDENCE_THRESH': 0.75,
                     'EVAL_CONFIG': {'IOU_THRESH': 0.5,
                                     'IOU_THRESHES': [0.5,
                                                      0.55,
                                                      0.6,
                                                      0.7,
                                                      0.75,
                                                      0.8,
                                                      0.85,
                                                      0.9,
                                                      0.95],
                                     'KEY_METRIC': 'MAP',
                                     'METRIC_NAMES': ['AP', 'MAP'],
                                     'OUTPUT_RAW_SCORE': False},
                     'NMS_CONFIG': {'nms_post_maxsize': 500,
                                    'nms_pre_maxsize': 4096,
                                    'nms_thresh

In [5]:
from pathlib import Path
from kitti.kitti_dataset import get_dataloader
cfg_path = Path("../kitti/cfg/kitti_dataset.yaml")
dataloader = get_dataloader(data_cfg_path=cfg_path, class_name_list=['Car'], batch_size=2, training=False)
for data in dataloader:
    test_data = data
    break
test_data = put_data_to_gpu(test_data)
test_data

{'frame_id': array(['007326', '007463'], dtype='<U6'),
 'calib': array([<kitti.io.calib_utils.Calibration object at 0x7f30c3665850>,
       dtype=object),
 'gt_boxes': tensor([[[ 3.6883e+00,  3.1541e+00, -9.6876e-01,  3.6900e+00,  1.7800e+00,
            1.5000e+00, -1.0796e-02,  1.0000e+00],
          [ 2.3860e+01, -1.0957e+01, -4.0633e-01,  3.6800e+00,  1.6400e+00,
            1.4400e+00, -1.6708e+00,  1.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[ 2.0899e+01,  2.9785e+00, -6.8543e-01,  4.0800e+00,  1.63

恢复模型：从模型文件恢复或模型权重文件恢复

In [6]:
from basic.utils.run.train import Predictor
# 从模型恢复
# model_path = '/home/ph/Desktop/PointCloud/utils_my/basic/utils/run/check_point/2021-11-16::16:32:50/best.pth'
# predictor = Predictor(model_path, class_names=['Car'])
# 从权重恢复
state_path = '/home/ph/Desktop/PointCloud/utils_my/basic/utils/run/check_point/2021-11-16::20:32:43/best.pth'
predictor = Predictor(detect_obj_type=['Car'], state_path=state_path, model_cfg=top_cfg)

先关闭后处理，直接观察Dense head的输出结果
 1. Dense Head输出的结果为：每个anchor对应的bbox以及每个anchor对应的分类得分

In [12]:
from copy import deepcopy
post_cfg = deepcopy(top_cfg.INFERENCE_CONFIG.POST_PROCESSING)
top_cfg.INFERENCE_CONFIG.POST_PROCESSING = None # 关闭后处理
post_cfg

{'CONFIDENCE_THRESH': 0.75,
 'EVAL_CONFIG': {'KEY_METRIC': 'MAP',
  'METRIC_NAMES': ['AP', 'MAP'],
  'IOU_THRESH': 0.5,
  'IOU_THRESHES': [0.5, 0.55, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95],
  'OUTPUT_RAW_SCORE': False},
 'NMS_CONFIG': {'nms_type': 'nms_gpu',
  'nms_thresh': 0.01,
  'num_topk': 4096,
  'nms_pre_maxsize': 4096,
  'nms_post_maxsize': 500}}

In [9]:
loss= predictor.predict_bbox(test_data, eval=False)
loss

{'tol_loss': tensor(0.0707, device='cuda:0', grad_fn=<AddBackward0>),
 'cls_loss': tensor(0.0034, device='cuda:0', grad_fn=<DivBackward0>),
 'reg_loss': tensor(0.0639, device='cuda:0', grad_fn=<DivBackward0>)}

In [13]:
preds_dict = predictor.predict_bbox(test_data, eval=True)
proposal_dict = preds_dict['proposal_dict']
print('proposals shape:', proposal_dict['proposals'].shape)

proposals shape: torch.Size([2, 70400, 7])


In [14]:
print('proposals scores shape:', proposal_dict['proposal_scores'].shape)
proposal_dict['proposal_scores']

proposals scores shape: torch.Size([2, 70400, 2])


tensor([[[0.9664, 0.0336],
         [0.9724, 0.0276],
         [0.9559, 0.0441],
         ...,
         [0.9631, 0.0369],
         [0.9561, 0.0439],
         [0.9532, 0.0468]],

        [[0.9664, 0.0336],
         [0.9724, 0.0276],
         [0.9559, 0.0441],
         ...,
         [0.9631, 0.0369],
         [0.9561, 0.0439],
         [0.9532, 0.0468]]], device='cuda:0', grad_fn=<SoftmaxBackward>)

绘制原始proposals中，Car分类得分高于0.75的bbox

In [15]:
%matplotlib auto
from kitti.io.kitti_io import KittiIo
file_reader = KittiIo(root="/home/ph/Dataset/KITTI/training")
for i in range(2):
    pos_mask = proposal_dict['proposal_scores'][i, :, 1] > 0.75 # Car得分高于0.75
    bbox = proposal_dict['proposals'][i, pos_mask]
    mask = test_data['points'][:, 0] == i
    w = VisualWindow(mode='3d', file_reader=file_reader)
    w.draw_point_cloud(test_data['points'][mask][:, 1:].cpu())
    w.draw_boxes3d(test_data['gt_boxes'][..., :-1][i].cpu().numpy(), 'corner')
    w.draw_boxes3d(bbox.cpu().detach().numpy(), 'corner', c='r')

Using matplotlib backend: Qt5Agg


# Post Process
 后处理：处理Dense head输出的原始proposals。主要有两个步骤
 1. NMS
 2. 计算AP和MAP

## NMS
 先恢复后处理配置
 1. perd bbox 与 gt的iou高于0.5就认为为正列
 2. 置信度阈值为0.75

In [16]:
top_cfg.INFERENCE_CONFIG['POST_PROCESSING'] = post_cfg
top_cfg.INFERENCE_CONFIG.POST_PROCESSING.EVAL_CONFIG.IOU_THRESH = 0.7
# top_cfg.INFERENCE_CONFIG.POST_PROCESSING.NMS_CONFIG.nms_thresh = 0.3
top_cfg.INFERENCE_CONFIG.POST_PROCESSING

{'CONFIDENCE_THRESH': 0.75,
 'EVAL_CONFIG': {'KEY_METRIC': 'MAP',
  'METRIC_NAMES': ['AP', 'MAP'],
  'IOU_THRESH': 0.7,
  'IOU_THRESHES': [0.5, 0.55, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95],
  'OUTPUT_RAW_SCORE': False},
 'NMS_CONFIG': {'nms_type': 'nms_gpu',
  'nms_thresh': 0.01,
  'num_topk': 4096,
  'nms_pre_maxsize': 4096,
  'nms_post_maxsize': 500}}

In [17]:
from basic.utils.nms_utils import NMS3D
nms = NMS3D(**top_cfg.INFERENCE_CONFIG.POST_PROCESSING.NMS_CONFIG,
            confidence_thresh=0.75)

### Multi classes NMS for Batch
 NMS的输出统一为列表字典，即list(dict1()，dict2()...,dictn()). 每个字典内保存一个场景、frame的NMS输出结果

In [18]:
nms_ret = nms.multi_classes_nms_for_batch(bbox_scores=proposal_dict['proposal_scores'],
                                          bbox_preds=proposal_dict['proposals'])

In [19]:
%matplotlib auto
for i in range(2):
    print("#" * 20, "frame:", test_data['frame_id'][i], "#" * 20)
    mask = test_data['points'][:, 0] == i
    points = test_data['points'][mask]
    bbox = nms_ret[i]['bboxes']
    scores = nms_ret[i]['scores']
    labels = nms_ret[i]['labels']
    print("number of predict bbox:", bbox.shape[0])
    print("scores:", scores)
    print("labels:", labels.T)
    w = VisualWindow(mode='3d', file_reader=file_reader)
    w.draw_point_cloud(points[:, 1:-1].cpu())
    w.draw_boxes3d(test_data['gt_boxes'][..., :-1][i].cpu().numpy(), 'corner')
    w.draw_boxes3d(bbox.cpu().detach().numpy(), 'corner', c='r')

Using matplotlib backend: Qt5Agg
#################### frame: 007326 ####################
number of predict bbox: 2
scores: tensor([0.9845, 0.8865], device='cuda:0', grad_fn=<IndexBackward>)
labels: tensor([[1., 1.]], device='cuda:0')
#################### frame: 007463 ####################
number of predict bbox: 7
scores: tensor([0.9908, 0.9640, 0.9561, 0.9402, 0.9098, 0.8992, 0.8170],
       device='cuda:0', grad_fn=<IndexBackward>)
labels: tensor([[1., 1., 1., 1., 1., 1., 1.]], device='cuda:0')


### Single class NMS for Batch

In [14]:
nms_ret1 = nms.single_class_nms_for_batch(cls_scores=proposal_dict['proposal_scores'][..., -1],
                                          bbox_preds=proposal_dict['proposals'])
nms_ret1

[{'bboxes': tensor([[ 1.4179e+01, -6.5328e+00, -7.5928e-01,  3.9185e+00,  1.6594e+00,
            1.5457e+00,  1.3594e+00],
          [ 2.1740e+01, -1.7935e+00, -7.5560e-01,  3.9412e+00,  1.6465e+00,
            1.5299e+00,  3.2808e-02],
          [ 1.3376e+01, -1.1369e+01, -7.6351e-01,  3.9117e+00,  1.6425e+00,
            1.5384e+00,  1.2233e+00],
          [ 3.6167e+01,  1.8038e+00, -7.4910e-01,  3.9217e+00,  1.6404e+00,
            1.5317e+00, -7.9402e-02],
          [ 5.1009e+01,  5.0521e+00, -7.5099e-01,  3.8713e+00,  1.6294e+00,
            1.5277e+00,  1.4147e+00]], device='cuda:0', grad_fn=<IndexBackward>),
  'scores': tensor([0.9903, 0.9745, 0.9436, 0.9208, 0.8245], device='cuda:0',
         grad_fn=<IndexBackward>)},
 {'bboxes': tensor([[38.9712, 13.8026, -0.7483,  3.9462,  1.6407,  1.5365, -0.1572]],
         device='cuda:0', grad_fn=<IndexBackward>),
  'scores': tensor([0.9061], device='cuda:0', grad_fn=<IndexBackward>)}]

### Multi Classes NMS for Frame

In [15]:
nms_ret2 = nms.multi_classes_nms_for_frame(bbox_scores=proposal_dict['proposal_scores'][1],
                                           bbox_pred=proposal_dict['proposals'][1])
nms_ret2

{'bboxes': tensor([[38.9712, 13.8026, -0.7483,  3.9462,  1.6407,  1.5365, -0.1572]],
        device='cuda:0', grad_fn=<CatBackward>),
 'scores': tensor([0.9061], device='cuda:0', grad_fn=<CatBackward>),
 'labels': tensor([[1.]], device='cuda:0')}

# Recall、Precision、AP
 利用NMS输出的结果计算每个场景的Recall、precision、AP、MAP
 1. 这里的MAP：每个场景中单一类别在不同IOU_THRESH下的平均AP
 2. 每个场景中全部类别的MAP的平均，称为MMAP

In [20]:
from basic.metric.average_precision import recall_and_precision, voc_ap, mean_ap
import numpy as np
for i in range(2):
    print()
    print("*" * 20, f"Frame:{test_data['frame_id'][i]}", "*" * 20)
    mask = test_data['points'][:, 0] == i
    points = test_data['points'][mask]
    bbox = nms_ret[i]['bboxes']
    scores = nms_ret[i]['scores']
    labels = nms_ret[i]['labels']
    print("scores:", scores)
    print("labels:", labels)
    real_mask = test_data['gt_boxes'][i][..., -1] > 0
    real_gts = test_data['gt_boxes'][i][real_mask][:, :-1]
    recall, pr = recall_and_precision(pred_bboxes=bbox, gts=real_gts, iou_thresh=0.5)
    ap = voc_ap(recall, pr)
    m_ap = mean_ap(pred_bboxes=bbox, gts=real_gts, iou_threshes=np.arange(0.5, 0.95, 0.05))
    print("recall:", recall)
    print("precision:", pr)
    print("AP:", ap)
    print("MAP:", m_ap)
    plt.plot(recall, pr)
    plt.show()


******************** Frame:007326 ********************
scores: tensor([0.9845, 0.8865], device='cuda:0', grad_fn=<IndexBackward>)
labels: tensor([[1.],
        [1.]], device='cuda:0')
recall: [0.  0.5]
precision: [0.  0.5]
AP: 0.24999999875
MAP: 0.02777777763888889

******************** Frame:007463 ********************
scores: tensor([0.9908, 0.9640, 0.9561, 0.9402, 0.9098, 0.8992, 0.8170],
       device='cuda:0', grad_fn=<IndexBackward>)
labels: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')
recall: [0.14285714 0.28571429 0.42857143 0.57142857 0.71428571 0.71428571
 0.71428571]
precision: [0.99999999 1.         1.         1.         1.         0.83333333
 0.71428571]
AP: 0.7142857128571429
MAP: 0.23492063445079367


完整的预测，包含Post Process

In [21]:
predictor = Predictor(detect_obj_type=['Car'], state_path=state_path, model_cfg=top_cfg)
final_dict = predictor.predict_bbox(test_data)
pprint(final_dict)

[{'bboxes': tensor([[ 23.7748, -10.9450,  -0.7620,   3.9227,   1.6540,   1.5439,   1.2276],
        [  3.7798,   3.4027,  -0.7485,   3.9461,   1.6369,   1.5294,  -0.1343]],
       device='cuda:0', grad_fn=<IndexBackward>),
  'eval_dict': {'AP': 0.0, 'MAP': 0.02777777763888889},
  'frame_id': '007326',
  'labels': tensor([[1.],
        [1.]], device='cuda:0'),
  'scores': tensor([0.9845, 0.8863], device='cuda:0', grad_fn=<IndexBackward>)},
 {'bboxes': tensor([[ 1.4588e+01,  7.0640e+00, -7.6193e-01,  3.9151e+00,  1.6595e+00,
          1.5482e+00,  1.4365e+00],
        [ 1.3754e+01, -2.5938e+00, -7.5852e-01,  3.9502e+00,  1.6483e+00,
          1.5303e+00, -6.0096e-02],
        [ 2.1355e+01,  3.0027e+00, -7.6004e-01,  3.9437e+00,  1.6492e+00,
          1.5361e+00,  2.9144e-03],
        [ 2.8956e+01,  2.6022e+00, -7.4678e-01,  3.9359e+00,  1.6433e+00,
          1.5373e+00, -1.4129e-02],
        [ 4.5765e+01, -2.5992e+00, -7.4511e-01,  3.9257e+00,  1.6401e+00,
          1.5370e+00, -3.0017e-